<a href="https://colab.research.google.com/github/gin614pham/Server_wav2vec2_vi/blob/main/trainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
model_checkpoint = "facebook/wav2vec2-large-xlsr-53"
batch_size = 16

In [ ]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("multi_lingual_speech_recognition_notebook", framework="pytorch")

In [ ]:
from datasets import load_dataset

In [ ]:
common_voice_train = load_dataset("mozilla-foundation/common_voice_11_0", "vi", split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_11_0", "vi", split="test")

README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...


Reading metadata...: 1237it [00:00, 86770.48it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11476it [00:00, 82490.58it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 337it [00:00, 99254.30it/s]


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,bà ấy nhìn em rất lâu rồi đáp lại
1,Trời đã về khuya dần
2,Nơi đây biết bao
3,Vậy Bây giờ chúng ta cần phải làm sao
4,Anh Hội suy nghĩ
5,Chừa cho cái tính tò mò đi nha
6,Dạ Cháu không có ý đó cháu chỉ là
7,tạm biệt anh
8,bài tập về nhà
9,Ghé qua quán nước của Hà Ni và nói với cô


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/2773 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,hà liền ấp úng đáp lại
1,còn nghe chúng nó đồn là cái chết của nó đáng sợ lắm
2,anh có thể dối em
3,là anh đây chỉ thăm hỏi đôi lời
4,mở cửa bước vào trong nhà
5,đến khi đi học
6,thì cả đời này phải gắn bó không được bỏ rơi
7,bởi quý trước tượng phật bà thích mình tội nghiệp quá nhiều
8,nhưng con chỉ khóc
9,khiến cho anh ta lúc ngủ cứ khẽ rùng mình lên vì sợ


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(
  extract_all_chars, batched=True,
  batch_size=-1, keep_in_memory=True,
  remove_columns=common_voice_train.column_names
)
vocab_test = common_voice_test.map(
  extract_all_chars, batched=True,
  batch_size=-1, keep_in_memory=True,
  remove_columns=common_voice_test.column_names
)

Map:   0%|          | 0/2773 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ú': 0,
 'e': 1,
 'ế': 2,
 'ử': 3,
 'ẩ': 4,
 'ì': 5,
 'ẹ': 6,
 'ỹ': 7,
 'ỉ': 8,
 'ề': 9,
 'b': 10,
 "'": 11,
 'ỷ': 12,
 'l': 13,
 'è': 14,
 'ẽ': 15,
 'd': 16,
 'ê': 17,
 'ữ': 18,
 'ó': 19,
 'ợ': 20,
 'ã': 21,
 'ừ': 22,
 'y': 23,
 'ỳ': 24,
 'f': 25,
 'ò': 26,
 'ớ': 27,
 'ũ': 28,
 ' ': 29,
 'ạ': 30,
 'ầ': 31,
 'ẫ': 32,
 'ẵ': 33,
 'm': 34,
 'ụ': 35,
 'õ': 36,
 'ễ': 37,
 'g': 38,
 'r': 39,
 'ỵ': 40,
 'ư': 41,
 'ồ': 42,
 'ủ': 43,
 'đ': 44,
 'c': 45,
 'ù': 46,
 'q': 47,
 'ị': 48,
 'ằ': 49,
 'n': 50,
 'k': 51,
 'ậ': 52,
 'h': 53,
 'ẳ': 54,
 'ứ': 55,
 'ặ': 56,
 'á': 57,
 's': 58,
 'ự': 59,
 'ờ': 60,
 'ở': 61,
 'ỡ': 62,
 'ắ': 63,
 'a': 64,
 'w': 65,
 'é': 66,
 't': 67,
 'ệ': 68,
 'à': 69,
 'ỗ': 70,
 'o': 71,
 'ộ': 72,
 'ỏ': 73,
 'í': 74,
 'z': 75,
 'i': 76,
 'ô': 77,
 'v': 78,
 'u': 79,
 'ơ': 80,
 'ý': 81,
 'j': 82,
 'â': 83,
 'ẻ': 84,
 'x': 85,
 'ổ': 86,
 'ể': 87,
 'ả': 88,
 'p': 89,
 'ọ': 90,
 'ă': 91,
 'ĩ': 92,
 'ố': 93,
 'ấ': 94}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

97

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  "./",
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.


In [ ]:
model_checkpoint_name = model_checkpoint.split("/")[-1]
repo_name = f"{model_checkpoint_name}-demo-colab"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-demo-colab/commit/95e714372f0b80487b8b4cfd44e3bfdfd6027fcd', commit_message='Upload tokenizer', commit_description='', oid='95e714372f0b80487b8b4cfd44e3bfdfd6027fcd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-demo-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='ginpham614/wav2vec2-large-xlsr-53-demo-colab'), pr_revision=None, pr_num=None)

In [ ]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/4d41f2253e68c139539b36607b20d6a01fe3cfb04b38792003f61c379c9d07d1/vi_train_0/common_voice_vi_23901117.mp3'

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/4d41f2253e68c139539b36607b20d6a01fe3cfb04b38792003f61c379c9d07d1/vi_train_0/common_voice_vi_23901117.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.90132641e-06, 7.18296724e-06, 1.22913943e-05]),
 'sampling_rate': 48000}

In [ ]:
from datasets import Audio


In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/4d41f2253e68c139539b36607b20d6a01fe3cfb04b38792003f61c379c9d07d1/vi_train_0/common_voice_vi_23901117.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.79643739e-05, -1.54107620e-05,  3.32880700e-06]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

định mệnh nào hoạ vẽ được tương lai 


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: tôi tin 
Input array shape: (35136,)
Sampling rate: 16000


In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/2773 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/2773 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
!pip install evaluate



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
  wer_metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained(
    model_checkpoint,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
if hasattr(model, "freeze_feature_extractor"):
  model.freeze_feature_extractor()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2178: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=batch_size,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

<ipython-input-59-ec2c74f250fd>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phucph-21it (phucph-21it-vku) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,17.897300,3.454554,1.000000
800,4.764000,1.451248,0.883734
1200,0.742100,1.355358,0.748931
1600,0.361800,1.464023,0.730394


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in 

TrainOutput(global_step=1800, training_loss=5.311797773573134, metrics={'train_runtime': 4997.4953, 'train_samples_per_second': 11.556, 'train_steps_per_second': 0.36, 'total_flos': 6.580913334631223e+18, 'train_loss': 5.311797773573134, 'epoch': 29.512396694214875})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-demo-colab/commit/9b3619a2cc6d2f548b8a6a0d0c473c3c78596799', commit_message='End of training', commit_description='', oid='9b3619a2cc6d2f548b8a6a0d0c473c3c78596799', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-demo-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='ginpham614/wav2vec2-large-xlsr-53-demo-colab'), pr_revision=None, pr_num=None)